<a href="https://colab.research.google.com/github/CatalystM47/Deep_Learning/blob/main/VGG16%2BReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt install unzip
!unzip hw4_doghole_keeper.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Archive:  hw4_doghole_keeper.zip
  inflating: train/bo/bo_1.jpg       
  inflating: train/bo/bo_10.jpg      
  inflating: train/bo/bo_11.jpg      
  inflating: train/bo/bo_12.jpg      
 extracting: train/bo/bo_13.jpg      
  inflating: train/bo/bo_14.jpg      
  inflating: train/bo/bo_15.jpg      
  inflating: train/bo/bo_16.jpg      
  inflating: train/bo/bo_17.jpg      
  inflating: train/bo/bo_18.jpg      
  inflating: train/bo/bo_19.jpg      
  inflating: train/bo/bo_2.jpg       
  inflating: train/bo/bo_3.jpg       
  inflating: train/bo/bo_4.jpg       
  inflating: train/bo/bo_5.jpg       
  inflating: train/bo/bo_6.jpg       
  inflating: train/bo/bo_7.jpg       
  inflating: train/bo/bo_8.jpg       
  inflating: train/bo/bo_9.jpg       
  inflating: train/not_bo/1.jp

In [6]:
import numpy as np
import cv2
import glob
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms as T
import torchvision.models as models

import sys
sys.setrecursionlimit(10000)

### 1. Data preparation

In [7]:
transform = T.Compose([T.Resize(256), T.RandomCrop(224), T.ToTensor()])
class Dataset(Dataset):
    def __init__(self, path='train'):
        super(Dataset, self).__init__()

        self.img_path = []
        bo_path = glob.glob(path + '/bo/*.jpg')
        notbo_path = glob.glob(path + '/not_bo/*.jpg')
        self.img_path = bo_path + notbo_path

        self.label_list = [1]*len(bo_path) + [0]*len(notbo_path)

    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])
        
        img_pil = Image.fromarray(img)
        self.img_tensor = transform(img_pil)

        self.label_tensor = torch.tensor(self.label_list[index])

        return self.img_tensor.to('cuda:0'), self.label_tensor.to('cuda:0')

    def __len__(self):
        return len(self.img_path)

In [8]:
training_dataset = Dataset('train')
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)

validation_dataset = Dataset('valid')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=False)

### 2. Constructing a convolutional neural network

In [9]:
#vgg16 = models.vgg16()
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
   param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [10]:
num_features = vgg16.classifier[0].in_features
vgg16.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)

In [11]:
vgg16 = vgg16.to('cuda:0')

### 3. Loss function and optimization method

In [12]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.0001)

### 4. Training of a neural network

In [13]:
for epoch in range(10):
    loss_val = 0

    for itr, data in enumerate(training_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = vgg16(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

Epoch: 1   , Loss: 5.371090192347765
Epoch: 2   , Loss: 0.4729023745749146
Epoch: 3   , Loss: 0.4402293391249259
Epoch: 4   , Loss: 0.07950235097814584
Epoch: 5   , Loss: 0.0281783010577783
Epoch: 6   , Loss: 0.03124845018464839
Epoch: 7   , Loss: 0.051500726832728105
Epoch: 8   , Loss: 0.014362634672579588
Epoch: 9   , Loss: 0.01446219440549612
Epoch: 10   , Loss: 0.016702202377359754


### 5. Prediction and evaluationfor the validation set

In [14]:
pred_list = []
label_list = []

for itr, data in enumerate(validation_loader):
    inputs, labels = data

    pred = vgg16(inputs)
    pred_category = torch.argmax(pred, dim=1)

    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list) == np.array(label_list) )
print("Val accu:", accu)


Val accu: 0.9333333333333333


@@ 딥러닝을 해야하는데 데이터셋이 너무 적을때 처리하는 방법 @@

1. !apt install unzip으로 압축풀기 프로그램 인스톨
2. Dataset Zip 파일 압축 풀기

1. 배열 사용 위해 numpy 이용
2. 이미지 처리 위해 cv2 이용
3. 원하는 파일 경로 찾기위해 glob이용
4. PIL == Python Image Library 파이썬으로 이미지 파일 사용 위해 이용

###(1)###
1. Dataset 클래스에서 다른 클래스로 init 상속위해 super이용
2. 이미지 경로 저장 위한 리스트 생성
3. bo_path에 bo 디렉터리 안의 모든 .jpg파일 경로 저장 (*은 모든 것을 뜻하는 asterisk)
4. not_bo도 동일
5. img path라는 모든 이미지의 경로를 포함한 리스트를 생성
6. bo인 이미지의 경로는 인덱스 [0]으로 라벨링, not_bo인 이미지의 경로는 인덱스[1]으로 라벨링
7. 아이템을 외부에서 가져오기 위해 getitem 생성.
이미지를 openCV 라이브러리를 이용하여 읽음, transform을 거친 것을 img_tensor로 저장
label_list들을 label_tensor로 저장 후 img_tensor와 label_tensor로 반환하는데, GPU메모리를 사용하기 위해
tensor.to('cuda:0')적용 (GPU Cuda 검색해보면 이해 가능)
8. 길이 return 위한 len 생성
이미지 path의 길이를 반환
9. transform을 사용하기위해 T.Compose 함수 사용 -> 이미지를 256size로 변경하고, 224size로 랜덤하게 잘라내며, 텐서 형태로 변경함

1. train에서 가져온 데이터를 데이터셋으로 만들어서 training_dataset으로 입력
2. 트레이닝을 위해 데이터 로더 생성, 데이터셋은 training_dataset을 사용하며, 배치 사이즈는 8개씩, 데이터를 무작위로 섞어 트레이닝
3. 검증위한 데이터 valid에서 가져온 데이터일 뿐, 나머지는 1과 같음
4. 검증을 위한 것이므로 섞을 필요는 없음

###(2)###
1. vgg16모델을 이용할 예정
2. vgg16모델의 형태를 확인
3. features의 숫자를 확인하기 위함
4. vgg16 classifier 정의. input feature를 256으로 줄이고 ReLU 실행, 최종 fully connected Layer의 결과가 2가 나오도록 설정.
5. vgg16을 cuda로 실행

###(3)###
1. Loss Function으로 Cross EntropyLoss사용
2. Optimizer는 Adam 사용 (Cross Entropy와 Adam은 찾아볼것), Learning_rate는 0.001 (LearningRate도 찾아보세요)

###(4)###
1. epoch(batch size만큼 나눠서 총 몇번 트레이닝 하는가)는 20으로 설정. LossVal = 0 으로 쓰레기 값 안들어가있게 초기화
2. 매 iteration에서는 training_loader를 자동으로 연산.
3. pred, loss, backward는 찾아보시는게 좋습니다. 특히 backward func는 역전파 함수로 알려져 있는데 중요해요.
4. (데이터가 PIL형식이 아니라, Numpy array라서 에러뜸) -> 기존에 numpy array형식으로 데이터가 들어간걸 컴퓨터가 기억하므로, 런타임을 초기화해서 기억 지우고 새로 시작하기 (새로 시작하면 PIL형식으로 데이터가 들어온걸로 인지함)
5. 첫번째 Epoch에서는 드라마틱하게 Loss가 줄어드나, 그 후로는 어느 이상 줄지 않음
6. Epoch 20 후 트레이닝 완료됨. 이제 검증데이터로 확인해야함

###(5)###
1. 검증을 하기위해 내가 모델을 통해 예측한 값을 넣을 pred_list 생성, 실제 값인 label_list 생성
2. validation loader를 연산한 값을 inputs, labels에 넣기
3. input (검증 데이터셋)을 vgg16으로 연산하여 예측한 값을 pred에 넣기
4. torch.argmax는 찾아보세요
5. 예측값 리스트 생성
6. 실제값 리스트 생성
7. accuray = 예측값 리스트와 실제값 리스트가 얼마나 같은지 수치값으로 정의
8. 정확도 출력.

###
처음으로 돌아가서 2번에 vgg16모델을 미리 트레이닝 시켜놓는 방법(pretraining) 이용한다는 것, 다시 pretraining한 후 
epoch를 10으로 줄여서 트레이닝 했음에도 불구하고, accuracy 확인하니 66%에서 90%로 훨씬 높아짐.
